# Deep Dream

The notebook contains the code from the book 'Deep Learning with Python'. This is a study reference and I would come back later to do coding modification for some more fancy images.

The algorithm generally exagerately activation layers of the pretrained deep network used for classification and combine the features extracted from thousands of real-life images to produce some images than could not exist in the human world, which works like creating the dream.

In [1]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


load the Keras package.

In [2]:
import keras
from keras.applications import inception_v3
from keras import backend as K

Using TensorFlow backend.


Load the pretrained Inception V3 model.

In [3]:
model = inception_v3.InceptionV3()
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
_____________________________________

Select several layer to exaggerate with a contribution value for each layer

In [0]:
# Set up the contributions You want for the selected activation layers
layer_coeffs = {'mixed2':1.,'mixed3':4.,'mixed4':5.,'mixed5':8.}
# The dictionary of all layers in the model
layer_dict = dict([(layer.name,layer) for layer in model.layers]) 

Set up the loss for the gradient ascent

In [5]:
loss = K.variable(0.)

for layer_name in layer_coeffs:
    print(layer_name)
    coeff = layer_coeffs[layer_name]
    activation = layer_dict[layer_name].output
    print('Shape of the activation: ',activation.shape)
    scaling = K.prod(K.cast(K.shape(activation),'float32'))
    loss = loss + coeff*K.sum(K.square(activation[:,2:-2,2:-2,:]))/scaling

mixed2
Shape of the activation:  (?, 35, 35, 288)
mixed3
Shape of the activation:  (?, 17, 17, 768)
mixed4
Shape of the activation:  (?, 17, 17, 768)
mixed5
Shape of the activation:  (?, 17, 17, 768)


Functions used for image rescaling, processing and gradient ascent.

In [0]:
Input = model.input

grads = K.gradients(loss,Input)[0]

grads = grads/K.maximum(K.mean(K.abs(grads)),1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([Input],outputs)

import scipy
from keras.preprocessing import image

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values
  
def gradient_ascent(x,iterations,step,max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('Loss Value at ',i,': ',loss_value)
        x += step*grad_values
    return x
  
def resize_img(img,size):
    img = np.copy(img)
    factors = (1,float(size[0])/img.shape[1],float(size[1])/img.shape[2],1)
    return scipy.ndimage.zoom(img,factors,order=1)

def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3,x.shape[2],x.shape[3]))
        x = x.transpose((1,2,0))
    else:
        x = x.reshape((x.shape[1],x.shape[2],3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x,0,255).astype('uint8')
    return x

def save_img(img,fname):
    pil_img = deprocess_image(np.copy(img))
    scipy.misc.imsave(fname,pil_img)
    
def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = inception_v3.preprocess_input(img)
    return img

Set up the hyperparameters and the image path. Doing some prework for the image transforming.

In [0]:
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 20.

base_image_path = '/content/drive/My Drive/Colab Notebooks/Thesis/data/dream/Test1.jpg'

img = preprocess_image(base_image_path)
original_shape = img.shape[1:3]
successive_shapes = [original_shape]

for i in range(1,num_octave):
    shape = tuple([int(dim/(octave_scale**i)) for dim in original_shape])
    successive_shapes.append(shape)
    
successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

Image transforming process.

In [8]:
for shape in successive_shapes:
    print('Processing image shape: ',shape)
    img = resize_img(img,shape)
    img = gradient_ascent(img,iterations=iterations,step=step,max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img,shape)
    same_size_original = resize_img(original_img,shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img
    
    img += lost_detail
    shrunk_original_img = resize_img(original_img,shape)
    save_img(img,fname='/content/drive/My Drive/Colab Notebooks/Thesis/data/dream/dream_at_scale_'+str(shape)+'.png')
    
save_img(img,fname='/content/drive/My Drive/Colab Notebooks/Thesis/data/dream/final_dream.png')

Processing image shape:  (459, 612)
Loss Value at  0 :  3.9896946
Loss Value at  1 :  5.116932
Loss Value at  2 :  6.633807
Loss Value at  3 :  8.180487
Loss Value at  4 :  9.760031
Loss Value at  5 :  11.361286
Loss Value at  6 :  12.935125
Loss Value at  7 :  14.485036
Loss Value at  8 :  16.010544
Loss Value at  9 :  17.48527
Loss Value at  10 :  18.985064


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:47: DeprecationWarning: `imsave` is deprecated!
`imsave` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imwrite`` instead.


Processing image shape:  (642, 857)
Loss Value at  0 :  6.067342
Loss Value at  1 :  8.868338
Loss Value at  2 :  11.370394
Loss Value at  3 :  13.575581
Loss Value at  4 :  15.625107
Loss Value at  5 :  17.596355
Loss Value at  6 :  19.384796


/usr/local/lib/python3.6/dist-packages/scipy/ndimage/interpolation.py:583: UserWarning: From scipy 0.13.0, the output shape of zoom() is calculated with round() instead of int() - for these inputs the size of the returned array has changed.
  "the returned array has changed.", UserWarning)


Processing image shape:  (900, 1200)
Loss Value at  0 :  6.051435
Loss Value at  1 :  8.717905
Loss Value at  2 :  11.057898
Loss Value at  3 :  13.204857
Loss Value at  4 :  15.234631
Loss Value at  5 :  17.193665
Loss Value at  6 :  19.027094
